# Imports

- stopword
- hyphen

(Target) 0-positive, 1-negative, 2-neutral

In [1]:
from keras import backend as K
from keras.layers import Reshape, Multiply
import time
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn.model_selection import KFold
import time

import numpy as np
from nltk import word_tokenize
%matplotlib inline
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D, DepthwiseConv2D, Conv2D, SeparableConv2D, MaxPooling1D, AveragePooling1D
from keras.layers import Input, concatenate, LeakyReLU
import gensim.downloader as api
from sklearn.model_selection import train_test_split
from keras.layers import Activation, Flatten, Dense, Dropout, SpatialDropout1D
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import GlobalAveragePooling2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Nadam, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.regularizers import l2
%config InlineBackend.figure_format='retina'
from keras_contrib.callbacks import CyclicLR
from keras.models import Model
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from data_science_utils.vision.keras import *
from time import time
import pandas as pd
from sklearn.metrics import f1_score
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import missingno as msno
import re
from joblib import Parallel, delayed
from data_science_utils import dataframe as df_utils
from data_science_utils import models as model_utils
from data_science_utils import plots as plot_utils
from data_science_utils.dataframe import column as column_utils
from data_science_utils import misc as misc
from data_science_utils import preprocessing as pp_utils
from data_science_utils import nlp as nlp_utils

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from data_science_utils.dataframe import get_specific_cols
from random import sample

import more_itertools
from more_itertools import flatten
import ast
from sklearn.preprocessing import LabelEncoder

import gc
from sklearn.decomposition import PCA

import sys
import os
sys.path.append(os.getcwd())
from importlib import reload


import warnings
warnings.filterwarnings('ignore')

reload(model_utils)
from data_science_utils.models import mean_absolute_percentage_error
from data_science_utils.models import median_absolute_percentage_error
from sklearn.metrics import mean_squared_error
import seaborn as sns
from more_itertools import flatten
import dill
from collections import Counter
import operator
from gensim.models import FastText
import itertools
from gensim.test.utils import datapath
from gensim.utils import tokenize
from gensim.models.fasttext import load_facebook_model

import lib
reload(lib)
from lib import *

import params
reload(params)
from params import *



Using TensorFlow backend.


[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw to /home/ec2-user/nltk_data...
[nltk_data]   Package omw is already up-to-date!


In [210]:
nlp_utils.stopwords_list = {}
nlp_utils.is_stopword('an')

False

In [2]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

df.shape
df_test.shape

(5279, 4)

(2924, 3)

# Preprocess

- Medical Vocab is very different from wiki/normal vocabs

In [3]:


df.text = df.text.apply(preprocess_string)
df_test.text = df_test.text.apply(preprocess_string)

df.drug = df.drug.apply(preprocess_string)
df_test.drug = df_test.drug.apply(preprocess_string)

## Find Occurences

In [4]:
df = find_occurence_count(df)
df_test = find_occurence_count(df_test)

We will later remove the sentences where drug name doesn't occur from training

In [5]:
df[df.occurences_count==0]
df_test[df_test.occurences_count==0]

,unique_hash,text,drug,sentiment,occurences,occurences_count
32,50b51148821b2d0eb23380ca8417b1d9fefd37af,had anyone been returned to a previous chemo ...,nivolumab,1,[],0
1856,ddb40991ffb9cddac847c77855977610bf4d03ae,any ideas anyone i m not due to see the doc...,opdivo,2,[],0
1973,ae868203b1cebf2c75722bc38c1f7611ad706ab0,be well,tarceva,1,[],0
2301,6df111aa3032069076b12f81bd87a9d72dc795ad,sandy,opdivo,2,[],0
2894,f275e928df77f875dc61f97c791b40519effc58c,i hope and pray you guys have the success i h...,opdivo,2,[],0
3446,f57ddcc186c8557ad4201bb252244bba749f23cf,any advice,opdivo,2,[],0
3525,65fb06982b3518557c8244b947fb36ae33b15534,chuck,opdivo,2,[],0
3720,c40d519b9b37c577ff6b385d3c639ef483525f07,there is hope,opdivo,2,[],0
4530,847039a8505f7a6f6db560a43009426204e8d835,all the best in the new year,opdivo,0,[],0
5118,2e0b9e15f5ae2fb7e6b5c7eecdff6a967b566a16,good luck with your wife and god bless,opdivo,0,[],0


,unique_hash,text,drug,occurences,occurences_count
421,d3fb8ee77b4a9797b6bd2d793429901be168ca64,these are immunolgy drugs and work against th...,opdivo,[],0
1862,231b3cc1744ebd896fa2420ba2ab7b7e35c8ab05,you have to keep taking it even in remission ...,opdivo,[],0
2542,a07303c7e50f033a7ec1c58bc2982949af63078d,3 wks later resumed low dose carbo x3 and 35...,tarceva,[],0


## Find Context

In [6]:
df["context_text"] = get_surrounding_text(df,surround)
df_test["context_text"] = get_surrounding_text(df_test,surround)

In [7]:
# df.occurences_count.value_counts() [df.occurences_count<10]

np.percentile(df_test['context_text'].apply(len).values,[50,90,95,99])

np.percentile(df['context_text'].apply(len).values,[50,90,95,99])

np.percentile(df_test['text'].apply(len).values,[50,90,95,99])

np.percentile(df['text'].apply(len).values,[50,90,95,99])



array([ 162.  ,  534.  ,  862.85, 2126.63])

array([ 157.  ,  514.2 ,  768.1 , 1994.92])

array([  860.5,  4714.3,  9219. , 27821. ])

array([  807.  ,  4191.6 ,  8327.5 , 21694.28])

In [8]:
df = preprocess_for_word_cnn(df, text_column='text', output_column="full_txt", word_length_filter=word_length_filter, jobs=jobs)
df_test = preprocess_for_word_cnn(df_test, text_column='text', output_column="full_txt", word_length_filter=word_length_filter, jobs=jobs)

df = preprocess_for_word_cnn(df, text_column='context_text', output_column="context_txt", word_length_filter=word_length_filter, jobs=jobs)
df_test = preprocess_for_word_cnn(df_test, text_column='context_text', output_column="context_txt", word_length_filter=word_length_filter, jobs=jobs)



100%|██████████| 2924/2924 [00:07<00:00, 370.99it/s]


## Review Length

In [9]:
df["review_length"] = df.full_txt.apply(len)/1000
df_test["review_length"] = df_test.full_txt.apply(len)/1000

df['review_length_by_occurences_count'] = df["review_length"]/(df['occurences_count']+1)
df_test['review_length_by_occurences_count'] = df_test["review_length"]/(df_test['occurences_count']+1)


In [10]:
df['last_occurence'] = df.occurences.apply(lambda x:x[-1] if len(x)>0 else -1)/1000
df['first_occurence'] = df.occurences.apply(lambda x:x[0] if len(x)>0 else -1)/1000

df_test['last_occurence'] = df_test.occurences.apply(lambda x:x[-1] if len(x)>0 else -1)/1000
df_test['first_occurence'] = df_test.occurences.apply(lambda x:x[0] if len(x)>0 else -1)/1000

df[["review_length","review_length_by_occurences_count","last_occurence","first_occurence"]].max()

review_length                        13.7350
review_length_by_occurences_count     6.8675
last_occurence                       64.9720
first_occurence                      64.7330
dtype: float64

## Mask (Extra Channel in CNN)

In [11]:
# arr = list(range(20))
# make_mask_single(arr,[5,6,12],3)

In [12]:
# text = "the quick brown fox jumps over the lazy dog woof bark".split(" ")
# make_mask_single(text,"over the lazy",3)

In [13]:
df = make_mask(df)
df_test = make_mask(df_test)



In [14]:
# list(zip(df.head(1).full_txt.values[0],df.head(1).full_txt_mask.values[0]))
# make_mask_single(df.head(1).full_txt.values[0],"gilenya",2)

## Per word Sentiment

In [15]:
# less frequent words must be given values towards 0 (neutral)
# also return occurence count of word
# return a apply_fn
def per_word_sentiment(df,text_col="",label_col=""):
    pos_counter = Counter()
    neg_counter = Counter()
    neural_counter = Counter()
    overall_counter = Counter()
    doc_counts = dict(df.sentiment.value_counts())
    total_docs = df.shape[0]
    pos_docs,neg_docs,neutral_docs = doc_counts[0],doc_counts[1],doc_counts[2]
    
    for row in df[[text_col,label_col]].values:
        t = set(row[0])
        lb = row[1]
        overall_counter.update(t)
        if lb==0:
            pos_counter.update(t)
        elif lb==1:
            neg_counter.update(t)
        elif lb==2:
            neural_counter.update(t)
        else:
            raise ValueError()
            
    w2s = {}
    counts = dict(overall_counter)
    total_pos_words = len(pos_counter)
    total_neg_words = len(neg_counter)
    total_words = len(overall_counter)
    
    for k,v in overall_counter.items():
        pos_oc = pos_counter[k]
        neg_oc = neg_counter[k]

        pos_frac = 100*pos_oc/pos_docs
        neg_frac = 100*neg_oc/neg_docs
        overall_frac = 100*v/total_docs
        sentiment = (pos_frac - neg_frac)/overall_frac
        # sentiment = (pos_oc - neg_oc)/v
        relative_trust = np.log10(v)*0.25
        sentiment = sentiment*relative_trust
        w2s[k] = sentiment
#         if overall_frac>2:
#             print(k,overall_frac)
        if k=='doctor':
            print(pos_oc,neg_oc,v)
            print(pos_frac,neg_frac,overall_frac,relative_trust,sentiment)
    def apply_w2s_fn(word_array):
        return list(map(lambda w:w2s[w] if w in w2s else 0.0,word_array))
    def apply_counts_fn(word_array):
        return list(map(lambda w:counts[w]/1000 if w in w2s else 0.0,word_array))
        
    return w2s,counts,apply_w2s_fn,apply_counts_fn
        
        
            
        
        

In [16]:
w2s,counts,apply_w2s_fn,apply_counts_fn = per_word_sentiment(df,"full_txt","sentiment")

68 140 824
11.021069692058347 16.726403823178018 15.609016859253646 0.7289818029242789 -0.2664539860960795


In [17]:
w = 'retinopathy'
counts[w]
w2s[w]
len(counts)
df[df.full_txt.apply(lambda x:w in x)]['sentiment'].value_counts()

apply_w2s_fn(["doctor","retinopathy"])

78

0.36587143013623225

31907

2    64
0    10
1     4
Name: sentiment, dtype: int64

[-0.2664539860960795, 0.36587143013623225]

In [18]:
df["word_sentiments"] = df.full_txt.apply(apply_w2s_fn)
df_test["word_sentiments"] = df_test.full_txt.apply(apply_w2s_fn)

df["word_count"] = df.full_txt.apply(apply_counts_fn)
df_test["word_count"] = df_test.full_txt.apply(apply_counts_fn)

df["word_sentiments_ctx"] = df.context_txt.apply(apply_w2s_fn)
df_test["word_sentiments_ctx"] = df_test.context_txt.apply(apply_w2s_fn)

df["word_count_ctx"] = df.context_txt.apply(apply_counts_fn)
df_test["word_count_ctx"] = df_test.context_txt.apply(apply_counts_fn)

In [19]:
np.max(df["word_sentiments"].apply(np.max))
np.max(df["word_count"].apply(np.max))

# np.max(df["word_sentiments_ctx"].apply(np.max))
# np.max(df["word_count_ctx"].apply(np.max))

1.2911763583107378

3.265

# Exploration

- How many text have 2 or more drug mention?
- How many reviews are used twice in dataset?
- Max, Min, Median, Mean sentence length
- Total number of words


In [20]:
df.columns

Index(['unique_hash', 'text', 'drug', 'sentiment', 'occurences',
       'occurences_count', 'context_text', 'full_txt', 'context_txt',
       'review_length', 'review_length_by_occurences_count', 'last_occurence',
       'first_occurence', 'full_txt_mask', 'full_txt_mask_gaussian',
       'word_sentiments', 'word_count'],
      dtype='object')

In [225]:
df.sentiment.value_counts()

2    3825
1     837
0     617
Name: sentiment, dtype: int64

In [226]:
np.percentile(df['full_txt'].apply(len),[90,95,99])
np.percentile(df_test['full_txt'].apply(len),[90,95,99])

array([ 414.  ,  831.2 , 2192.56])

array([ 461.1 ,  922.55, 2748.69])

In [22]:
df.groupby(["sentiment"])["review_length"].agg(['mean',"median","min","max"])

,mean,median,min,max
sentiment,,,,
0,180.581848,66,3,4009
1,135.354839,70,1,1844
2,225.993203,81,1,13735


In [24]:
# Drug overlap in train test set
len(set(df.drug.values))
len(set(df_test.drug.values))
set(df_test.drug.values) - set(df.drug.values)

102

95

{'afainib',
 'alitma',
 'duvalumab',
 'elotinib',
 'etrolizumab',
 'flixabi',
 'osmertinib',
 'risankizumab',
 'stellara'}

In [21]:
# How many words in test set are not in train set

train_words = set(list(more_itertools.flatten(df.full_txt.values)))
test_words = set(list(more_itertools.flatten(df_test.full_txt.values)))
all_words = set(list(more_itertools.flatten(df.full_txt.values))+list(more_itertools.flatten(df_test.full_txt.values)))
len(train_words)
len(test_words)
test_exlusive_words = test_words-train_words

print("Test Set Exclusive Distinct words = ",len(test_exlusive_words))

print("Total vocab size = ",len(all_words))


31907

24571

Test Set Exclusive Distinct words =  5618
Total vocab size =  37525


# Building Fasttext Model

In [ ]:
fname = os.getcwd()+"/fasttext.model"


In [42]:
crawl = datapath(os.getcwd()+"/crawl-300d-2M-subword.bin")
fb_model = load_facebook_model(crawl)


In [81]:

success = 0
fail = 0
for w in all_words:

    if w in fb_model.wv:
        success = success + 1
    else:
        fail = fail + 1

print("Words in FB Fasttext Model = ",success)
print("Words Not in FB Fasttext Model = ", fail)


Words in FB Fasttext Model =  37525
Words Not in FB Fasttext Model =  0


In [44]:
sentences = list(df.full_txt.values) + list(df_test.full_txt.values)
len(sentences)

8203

In [45]:
fb_model.build_vocab(sentences=sentences, update=True)

In [48]:
fb_model.corpus_total_words
fb_model.corpus_count

8203000

8203

In [49]:

fb_model.train(sentences=sentences, 
               total_examples=fb_model.corpus_count, epochs=1, total_words=fb_model.corpus_total_words)
fb_model.save(fname)

In [ ]:
from gensim.models import FastText
fb_model = FastText.load(fname)

# Encoding, Padding and Saving

## Encoding Labels


In [20]:
df.columns

df['ohe_labels'] = list(np_utils.to_categorical(df['sentiment'], 3))
df['ohe_labels'] = df['ohe_labels'].apply(lambda x:list(map(int,x)))

Index(['unique_hash', 'text', 'drug', 'sentiment', 'occurences',
       'occurences_count', 'context_text', 'full_txt', 'context_txt',
       'review_length', 'review_length_by_occurences_count', 'last_occurence',
       'first_occurence', 'full_txt_mask', 'full_txt_mask_gaussian',
       'word_sentiments', 'word_count', 'word_sentiments_ctx',
       'word_count_ctx'],
      dtype='object')

In [21]:
df_copy = df.copy()
df_test_copy = df_test.copy()

In [55]:
# df = df_copy.copy()
# df_test = df_test_copy.copy()

## Encoding and Padding

In [22]:
def do_padding(df):
    df['full_txt'] = list(pad_text_sequences(df['full_txt'].values, maxlen=full_txt_maxlen,jobs=jobs))
    df['context_txt'] = list(pad_text_sequences(df['context_txt'].values, maxlen=full_txt_maxlen,jobs=jobs)) # use of full_txt_maxlen is intentional
    # we do this since we will filter out in Keras model lambda layer

    df['full_txt_mask'] = list(pad_text_sequences(df['full_txt_mask'].values,empty=0.0, maxlen=full_txt_maxlen,jobs=jobs))
    df['full_txt_mask_gaussian'] = list(pad_text_sequences(df['full_txt_mask_gaussian'].values,empty=0.0, maxlen=full_txt_maxlen,jobs=jobs))
    
    
    df['word_sentiments'] = list(pad_text_sequences(df['word_sentiments'].values,empty=0.0, maxlen=full_txt_maxlen,jobs=jobs))
    df['word_count'] = list(pad_text_sequences(df['word_count'].values,empty=0.0, maxlen=full_txt_maxlen,jobs=jobs))
    df['word_sentiments_ctx'] = list(pad_text_sequences(df['word_sentiments_ctx'].values,empty=0.0, maxlen=full_txt_maxlen,jobs=jobs))
    df['word_count_ctx'] = list(pad_text_sequences(df['word_count_ctx'].values,empty=0.0, maxlen=full_txt_maxlen,jobs=jobs))
    
    return df


In [23]:
print("full text length = ",full_txt_maxlen)
df = do_padding(df)
df_test = do_padding(df_test)


full text length =  500


 17%|█▋        | 908/5279 [00:00<00:00, 9077.26it/s]

Total Words before Min frequency filtering 29815
Total Words after Min frequency filtering 11859


100%|██████████| 2924/2924 [00:00<00:00, 9326.42it/s] 


## Saving

In [24]:
df.to_csv("train_padded.csv",index=False)
df_test.to_csv("test_padded.csv",index=False)

In [25]:
df_copy.to_csv("train_non_padded.csv",index=False)
df_test_copy.to_csv("test_non_padded.csv",index=False)

In [26]:
df.columns
df_copy.columns

Index(['unique_hash', 'text', 'drug', 'sentiment', 'occurences',
       'occurences_count', 'context_text', 'full_txt', 'context_txt',
       'review_length', 'review_length_by_occurences_count', 'last_occurence',
       'first_occurence', 'full_txt_mask', 'full_txt_mask_gaussian',
       'word_sentiments', 'word_count', 'word_sentiments_ctx',
       'word_count_ctx', 'ohe_labels', 'full_txt_encoded',
       'context_txt_encoded'],
      dtype='object')

Index(['unique_hash', 'text', 'drug', 'sentiment', 'occurences',
       'occurences_count', 'context_text', 'full_txt', 'context_txt',
       'review_length', 'review_length_by_occurences_count', 'last_occurence',
       'first_occurence', 'full_txt_mask', 'full_txt_mask_gaussian',
       'word_sentiments', 'word_count', 'word_sentiments_ctx',
       'word_count_ctx', 'ohe_labels'],
      dtype='object')